In [1]:
# torch와 matplotlib 충돌 방지
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# image/label check

In [6]:
# 공식 ImageNet 홈페이지에서 다운받은 ImageNet2012 이미지 
import os

files = os.listdir('ILSVRC2012_img_val')
len(files)

50000

In [30]:
## new labels
## ImageNet에서 제공받은 label이 모델과 다르게 indexing 되어 있어
## 새로운 label 파일을 다운받음 https://gist.github.com/ksimonyan/fd8800eeb36e276cd6f9#note

with open('caffe_ilsvrc12.tar/val.txt') as f:
    
    text = f.read()

labels = [int(label.split('JPEG')[1]) for label in text.split('\n')[:-1]]
labels

[65,
 970,
 230,
 809,
 516,
 57,
 334,
 415,
 674,
 332,
 109,
 286,
 370,
 757,
 595,
 147,
 108,
 23,
 478,
 517,
 334,
 173,
 948,
 727,
 23,
 846,
 270,
 167,
 55,
 858,
 324,
 573,
 150,
 981,
 586,
 887,
 32,
 398,
 777,
 74,
 516,
 756,
 129,
 198,
 256,
 725,
 565,
 167,
 717,
 394,
 92,
 29,
 844,
 591,
 358,
 468,
 259,
 994,
 872,
 588,
 474,
 183,
 107,
 46,
 842,
 390,
 101,
 887,
 870,
 841,
 467,
 149,
 21,
 476,
 80,
 424,
 159,
 275,
 175,
 461,
 970,
 160,
 788,
 58,
 479,
 498,
 369,
 28,
 487,
 50,
 270,
 383,
 366,
 780,
 373,
 705,
 330,
 142,
 949,
 349,
 473,
 159,
 872,
 878,
 201,
 906,
 70,
 486,
 632,
 608,
 122,
 720,
 227,
 686,
 173,
 959,
 638,
 646,
 664,
 645,
 718,
 483,
 852,
 392,
 311,
 457,
 352,
 22,
 934,
 283,
 802,
 553,
 276,
 236,
 751,
 343,
 528,
 328,
 969,
 558,
 163,
 328,
 771,
 726,
 977,
 875,
 265,
 686,
 590,
 975,
 620,
 637,
 39,
 115,
 937,
 272,
 277,
 763,
 789,
 646,
 213,
 493,
 647,
 504,
 937,
 687,
 781,
 666,
 583,
 158

In [7]:
# 0~999까지의 값을 갖는 label 5만개
min(labels), max(labels), len(labels)

(0, 999, 50000)

# 현재 ImageNet에서 제공하는 label index와 torch에서 제공하는 pretrained model의 label index가 달라 맞추어주어야함
# --> pretrained model에 맞는 label로 다시 매핑함

# Make directory for classification

In [31]:
original_path = 'ILSVRC2012_img_val'
img_path = 'ImageNet_validation'

#새로운 폴더 생성
os.mkdir(img_path)

#class 별 폴더 생성
for i in range(1000):
    os.mkdir(img_path+'/'+str(i))
    
import shutil
from tqdm.notebook import tqdm

# 만들어둔 class별 폴더로 이미지를 복사함
for file, label in tqdm(zip(sorted(files), labels), total=50000):
    
    shutil.copyfile(original_path+'/'+file, img_path+'/'+str(label)+'/'+file)
    

  0%|          | 0/50000 [00:00<?, ?it/s]

# DataLoad

In [32]:
import torch
import torchvision
from torchvision import transforms as T

# https://github.com/pytorch/examples/blob/42e5b996718797e45c46a25c55b031e6768f8440/imagenet/main.py#L89-L101
# pytorch examples를 참조하여 전처리함
normalize = T.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])

transform = T.Compose([T.Scale(256), 
                       T.CenterCrop(224), 
                       T.ToTensor(),
                       normalize])

# ImageFolder를 사용하여 데이터를 불러온 후 DataLoader로 변환
val_data = torchvision.datasets.ImageFolder(root = img_path, transform=transform, )
val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, drop_last=False, shuffle=False)

len(val_data), len(val_loader)

(50000, 391)

# Model Load

In [33]:
# AlexNet
# VGG16
# ResNet18
# GoogLeNet


import torch
import torchvision

# pre-trained model 불러오기
alexnet = torchvision.models.alexnet(pretrained=True)
vgg16 = torchvision.models.vgg16(pretrained=True)
resnet18 = torchvision.models.resnet18(pretrained=True)
googlenet = torchvision.models.googlenet(pretrained=True)

# Model evaluate

In [100]:

def evaluate(model):
    
    global evals, pred, label, img, pred_idx
    
    evals = []
    model.eval()
    class_to_idx = val_data.class_to_idx
    
    with torch.no_grad():

        for img, label in tqdm(val_loader, total=len(val_loader)):
            
            #예측후 top1만 추출
            pred = model(img)
            pred = torch.max(pred, 1)[1]
            
            #원래 모델에 맞는 index로 바꾸어줌
            pred_idx = [class_to_idx[str(p)] for p in pred.numpy()]
            pred_idx = torch.tensor(pred_idx)
            
            # 평가
            evals.append(np.array(pred_idx == label))
            
#             if len(evals) > 2:
#                 break
        
    evals = np.concatenate(evals)
    
    return evals

In [101]:
alexnet_eval = evaluate(alexnet)

alexnet_eval.mean()

  0%|          | 0/391 [00:00<?, ?it/s]

0.56522

In [102]:
import joblib

joblib.dump(alexnet_eval, 'alexnet_eval')

['alexnet_eval']

In [107]:
vgg16_eval = evaluate(vgg16)

vgg16_eval.mean()

  0%|          | 0/391 [00:00<?, ?it/s]

0.71592

In [109]:
resnet18_eval = evaluate(resnet18)
resnet18_eval.mean()

  0%|          | 0/391 [00:00<?, ?it/s]

0.69758

In [120]:
joblib.dump(vgg16_eval, 'vgg16_eval')

['vgg16_eval']

In [116]:
googlenet_eval = evaluate(googlenet)
googlenet_eval.mean()

  0%|          | 0/391 [00:00<?, ?it/s]

0.69778

In [119]:
joblib.dump(resnet18_eval, 'resnet18_eval')

['resnet18_eval']

In [118]:
joblib.dump(googlenet_eval, 'googlenet_eval')

['googlenet_eval']

# Summary

In [9]:
import pandas as pd
import joblib

alexnet_eval = joblib.load('alexnet_eval')
vgg16_eval = joblib.load('vgg16_eval')
resnet18_eval = joblib.load('resnet18_eval')
googlenet_eval = joblib.load('googlenet_eval')


pd.DataFrame(index=['AlexNet', 'VGG16', 'ResNet18', 'GoogleNet'],
            columns=['Top-1 Accuracy'],
            data = [[alexnet_eval.mean()],
                      [vgg16_eval.mean()],
                      [resnet18_eval.mean()],
                      [googlenet_eval.mean()]
                     ])

,Top-1 Accuracy
AlexNet,0.56522
VGG16,0.71592
ResNet18,0.69758
GoogleNet,0.69778
